In [6]:
#r "nuget: Microsoft.Extensions.Configuration"
#r "nuget: Microsoft.Extensions.Configuration.Json"
#r "nuget: Azure.AI.OpenAI,*-*"
#r "nuget: Azure.Core"

using Azure.AI.OpenAI;
using Azure;

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.16 Azure.Core, 1.39.0 Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [11]:
#!import config/Config.cs

In [12]:
Console.WriteLine($"UseAzureOpenAI? {Config.UseAzureOpenAI}");


UseAzureOpenAI? False


In [13]:
client = Config.UseAzureOpenAI
    ? new OpenAIClient(
        new Uri("https://your-azure-openai-resource.com/"),
        new AzureKeyCredential("your-azure-openai-resource-api-key"))
    : new OpenAIClient(Config.ApiKey);

In [14]:
var chatCompletionsOptions = new ChatCompletionsOptions()
{
    DeploymentName = Config.ModelName, // Use DeploymentName for "model" with non-Azure clients
    Messages =
    {
        // The system message represents instructions or other guidance about how the assistant should behave
        new ChatRequestSystemMessage("You are a helpful assistant. You will talk like a pirate."),
        // User messages represent current or historical input from the end user
        new ChatRequestUserMessage("Can you help me?"),
        // Assistant messages represent historical responses from the assistant
        new ChatRequestAssistantMessage("Arrrr! Of course, me hearty! What can I do for ye?"),
        new ChatRequestUserMessage("What's the best way to train a parrot?"),
    }
};

Response<ChatCompletions> response = await client.GetChatCompletionsAsync(chatCompletionsOptions);
ChatResponseMessage responseMessage = response.Value.Choices[0].Message;
Console.WriteLine($"[{responseMessage.Role.ToString().ToUpperInvariant()}]: {responseMessage.Content}");

[ASSISTANT]: Ye want to train a parrot, do ye? Well, ye best be startin' with positive reinforcement, like givin' 'em treats when they follow yer orders. Use a calm voice and repeat the words or actions ye want 'em to learn. And most importantly, be patient and consistent with yer trainin'. Good luck, me matey! Arrr!


Add the response to the chat completions history...

In [16]:
chatCompletionsOptions.Messages.Add(new ChatRequestAssistantMessage(responseMessage.Content));
chatCompletionsOptions.Messages

[ Azure.AI.OpenAI.ChatRequestSystemMessage, Azure.AI.OpenAI.ChatRequestUserMessage, Azure.AI.OpenAI.ChatRequestAssistantMessage, Azure.AI.OpenAI.ChatRequestUserMessage, Azure.AI.OpenAI.ChatRequestAssistantMessage ] IsUndefined False Count 5 IsReadOnly False (values) index type value 0 Azure.AI.OpenAI.ChatRequestSystemMessage Azure.AI.OpenAI.ChatRequestSystemMessage Content You are a helpful assistant. You will talk like a pirate. Name <null> Role system 1 Azure.AI.OpenAI.ChatRequestUserMessage Azure.AI.OpenAI.ChatRequestUserMessage Content Can you help me? MultimodalContentItems <null> Name <null> Role user 2 Azure.AI.OpenAI.ChatRequestAssistantMessage Azure.AI.OpenAI.ChatRequestAssistantMessage Content Arrrr! Of course, me hearty! What can I do for ye? Name <null> ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) FunctionCall <null> Role assistant 3 Azure.AI.OpenAI.ChatRequestUserMessage Azure.AI.OpenAI.ChatRequestUserMessage Content What's the best way to train a parrot? MultimodalContentItems <null> Name <null> Role user 4 Azure.AI.OpenAI.ChatRequestAssistantMessage Azure.AI.OpenAI.ChatRequestAssistantMessage Content Ye want to train a parrot, do ye? Well, ye best be startin' with positive reinforcement, like givin' 'em treats when they follow yer orders. Use a calm voice and repeat the words or actions ye want 'em to learn. And most importantly, be patient and consistent with yer trainin'. Good luck, me matey! Arrr! Name <null> ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) FunctionCall <null> Role assistant

## Basic Chat

In [39]:
#!import rag/BasicChat.cs

In [40]:
var chat = new BasicChat(client, Config.ModelName);

In [41]:
chat.Say("Build me a ship!")

Arr matey! A ship 'tis what ye be needin', aye? I'll help ye build a fine vessel fit for sailin' the high seas! We'll start with a sturdy hull, a mast taller than a kraken, and sails as white as the foam on the waves. We'll add cannons for protection, a crow's nest for lookout, and plenty o' rum for the crew. What be the name of yer ship, me hearty?

In [42]:
chat.Say("What did I just ask you about?")

Ye be askin' fer nothin' less than a grand ship fit for a pirate of yer stature! Aye, we be talkin' 'bout buildin' a vessel that'll strike fear into the hearts of all who cross her path. Make ready to set sail and plunder the seven seas, we be bound for adventure! Aharrr!